# Setup der Datenbanken
### In diesem Notebook wird eine Verbindung zu unserer Azure MySQL Datenbank hergestellt. Anschließend werden die für das BI Tool nötigen Tabellen angelegt. Als erstes wird das Package mysql-connector-python installiert.

In [1]:
!pip install mysql-connector-python
!pip install python-dotenv

### Als nächstes stellen wir eine Verbindung zu unserer Datenbank her und legen einen Cursor für diese an.

In [1]:
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()
username = os.environ.get('AZURE_USER')
password = os.environ.get('PASSWORD')
host = os.environ.get('HOST')
db = os.environ.get('DB')

# Establish the connection
conn = mysql.connector.connect(
  user=username, 
  password=password, 
  host=host, 
  database=db
)
print(conn)

In [2]:
cursor = conn.cursor()

### Danach werden die 4 Tabellen Ernte, Feld, Niederschlag_Pro_Tag und Wasserbedarfsanalyse erstellt.

In [3]:
cursor.execute("CREATE TABLE ernte (ernte_id INT PRIMARY KEY AUTO_INCREMENT, ernte_name VARCHAR(100), wasserbedarf_prognose INT)")

In [4]:
cursor.execute("CREATE TABLE feld (feld_id INT PRIMARY KEY AUTO_INCREMENT, feld_groesse INT, lat FLOAT, lon FLOAT, ernte_id INT, FOREIGN KEY (ernte_id) REFERENCES ernte(ernte_id))")

In [5]:
cursor.execute("CREATE TABLE niederschlag_pro_tag (datum VARCHAR(12), niederschlag FLOAT, feld_id INT, FOREIGN KEY (feld_id) REFERENCES feld(feld_id))")

In [6]:
cursor.execute("CREATE TABLE wasserbedarfsanalyse (feld_id INT NOT NULL, ernte_id INT NOT NULL, datum VARCHAR(12) NOT NULL, wasserbedarf_pro_tag FLOAT NOT NULL, wasserbedarf_pro_tag_pro_m2 FLOAT NOT NULL, FOREIGN KEY (feld_id) REFERENCES feld(feld_id), FOREIGN KEY (ernte_id) REFERENCES ernte(ernte_id), PRIMARY KEY(feld_id, ernte_id, datum))")

### Unsere Ernte-Tablle wird mit den ungefähren Wasserbedarfswerten befüllt. Diese sind hierbei Schätzungen für ( LITER / M² ) / TAG

In [7]:
sql_code = "INSERT INTO ernte (ernte_name, wasserbedarf_prognose) VALUES (%s, %s)"

ernte_values = [
    ("rice", 4),
    ("maize", 17),
    ("Soyabeans", 12),
    ("beans", 13),
    ("peas", 17),
    ("groundnuts", 2),
    ("cowpeas", 6),
    ("banana", 5),
    ("mango", 3),
    ("grapes", 8),
    ("watermelon", 7),
    ("apple", 4),
    ("orange", 9),
    ("cotton", 1),
    ("coffee", 6)
]

cursor.executemany(sql_code, ernte_values)
conn.commit()

In [8]:
cursor.execute("SELECT * FROM ernte")
result = cursor.fetchall()

for x in result:
    print(x)

(1, 'rice', 4)
(2, 'maize', 17)
(3, 'Soyabeans', 12)
(4, 'beans', 13)
(5, 'peas', 17)
(6, 'groundnuts', 2)
(7, 'cowpeas', 6)
(8, 'banana', 5)
(9, 'mango', 3)
(10, 'grapes', 8)
(11, 'watermelon', 7)
(12, 'apple', 4)
(13, 'orange', 9)
(14, 'cotton', 1)
(15, 'coffee', 6)


In [7]:
#cursor.execute("DROP TABLE wasserbedarfsanalyse")
#cursor.execute("DROP TABLE niederschlag_pro_tag")
#cursor.execute("DROP TABLE feld")
#cursor.execute("DROP TABLE ernte") 